# Nội dung chính:
1. Xây dựng mạng Neural với Raw Tensorflow.
2. Bài tập

# 1. Xây dựng mạng neural với Raw Tensorflow

Chúng ta sẽ xây dựng một mạng neural gồm 2 lớp ẩn với Tensorflow để xử lý bài toàn quen thuộc là MNIST.

Tập dữ liệu là bộ chữ số viết tay MNIST. Dataset bao gồm 60000 mẫu train và 10000 mẫu test. Các hình ảnh có kích thước cố định là 28x28 và chuẩn hoá giá trị từ 0 đến 1. Để đơn giản, mỗi hình ảnh đã được làm phẳng (flatten) và chuyển đổi thành một mảng numpy 1-D gồm 784 đặc trưng (28 * 28).

Quá trình xây dựng mạng nơ ron sẽ trải qua 4 bước:

1. Chuẩn bị dữ liệu đầu vào.
2. Xây dựng mạng nơ ron.
3. Thiết lập hàm loss function và phương pháp tối ưu gradient descent.
4. Fitting mô hình và đánh giá kết quả.

Các bước lần lượt như bên dưới:

## 1.1 Chuẩn bị dữ liệu

In [1]:
import os
import gzip
import shutil
import struct
import urllib
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

def parse_data(path, dataset, flatten):
    if dataset != 'train' and dataset != 't10k':
        raise NameError('dataset must be train or t10k')

    label_file = os.path.join(path, dataset + '-labels-idx1-ubyte')
    with open(label_file, 'rb') as file:
        _, num = struct.unpack(">II", file.read(8))
        labels = np.fromfile(file, dtype=np.int8)  # int8
        new_labels = np.zeros((num, 10))
        new_labels[np.arange(num), labels] = 1

    img_file = os.path.join(path, dataset + '-images-idx3-ubyte')
    with open(img_file, 'rb') as file:
        _, num, rows, cols = struct.unpack(">IIII", file.read(16))
        imgs = np.fromfile(file, dtype=np.uint8).reshape(num, rows, cols)  # uint8
        imgs = imgs.astype(np.float32) / 255.0
        if flatten:
            imgs = imgs.reshape([num, -1])

    return imgs, new_labels

def read_mnist(path, flatten=True, num_train=55000):
    """
    Read in the mnist dataset, given that the data is stored in path
    Return two tuples of numpy arrays
    ((train_imgs, train_labels), (test_imgs, test_labels))
    """
    imgs, labels = parse_data(path, 'train', flatten)
    indices = np.random.permutation(labels.shape[0])
    train_idx, val_idx = indices[:num_train], indices[num_train:]
    train_img, train_labels = imgs[train_idx, :], labels[train_idx, :]
    val_img, val_labels = imgs[val_idx, :], labels[val_idx, :]
    test = parse_data(path, 't10k', flatten)
    return [train_img, train_labels], [val_img, val_labels], test

mnist_folder = './data/mnist'
train, val, test = read_mnist(mnist_folder, flatten=True)

Instructions for updating:
non-resource variables are not supported in the long term


## 1.2 Xây dựng mạng neural

In [2]:
# tham số mạng
n_hidden_1 = 256 # số feature layer 1
n_hidden_2 = 256
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10  # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

X và Y lần lượt là 2 placeholder được sử dụng để chứa biến dự báo và và biến được dự báo. Kích thước của nó có height bằng None ngụ ý rằng ta có thể đưa vào bao nhiêu quan sát tùy ý. Điều này thuận lợi cho fitting model theo các batch_size khác nhau ở bước sau.

Ta nhận thấy các giá trị cần điều chỉnh (tuning) trong mô hình là các weight và bias. Do đó trong mạng neural ta sẽ xây dựng các layer dựa trên ma trận hệ số và vector bias. Lưu ý chúng ta phải khai báo các ma trận hệ số và vector bias dưới dạng Variable để thuật toán tối ưu có thể hiểu được đây là các giá trị cần cập nhật. 

In [3]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [4]:
# tạo model
def neural_network(x):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# 1.3 Thiết lập hàm mất mát và optimizer

In [5]:
learning_rate = 0.1
# Kiến trúc mô hình
logits = neural_network(X)

# định nghịa loss và optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# khởi tạo tất cả các biến
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Hàm `tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels)` sẽ tính ra hàm ra giá trị cross entropy giữa 2 phân phối được dự báo từ mạng nơ ron (logits) và giá trị thực tế (ground truth). Giá trị này càng nhỏ thì giữa 2 phân phối càng sát nhau tức mô hình đưa ra dự báo càng chuẩn xác. Thuật toán gradient descent mà chúng ta sử dụng là AdamOptimizer với learning_rate = 0.1. Lưu ý khi ta kích hoạt quá trình train_op thì thuật toán AdamOptimizer sẽ tự động tìm đến các variable và cập nhật lại variable theo phương gradient descent. 

Do đó các weights và bias của mỗi layer đều được cập nhật sau mỗi lượt huấn luyện. Tuy nhiên có một số trường hợp thuật toán sẽ dừng lại khi gradient quá nhỏ và accuracy và loss function dường như không thay đổi. Những trường hợp này ta cần thay đổi tốc độ học, thay đổi hàm loss function hoặc điều chỉnh lại cấu trúc mạng nơ ron.

Hàm `tf.argmax(x, 1)` sẽ tìm ra class có xác xuất lớn nhất trong lớp các class trả về. Đây chính là class được dự báo. Hàm `tf.equal()` sẽ so sánh class dự báo có trùng với class thực tế (ground truth) hay không. acc là tỷ lệ phần trăm dự báo chính xác được tính ra từ trung bình của toàn bộ các kết quả so sánh ở toàn bộ các quan sát.

## 1.4 Fitting model

In [6]:
num_steps = 500
batch_size = 128
display_step = 50

# tạo hàm lấy batch tiếp theo. Khi lấy hết batch của mẫu thì shuffle lại mẫu.
def next_batch(X, batch_size, index=0):
    start = index
    index += batch_size
    if index > len(X[0]):
        perm = np.arange(len(X[0]))
        np.random.shuffle(perm)
        X[0] = X[0][perm]
        X[1] = X[1][perm]
        start = 0
        index = batch_size
    end = index
    return X[0][start:end], X[1][start:end], index

In [7]:
# Start training
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    idx = 0
    for step in range(1, num_steps + 1):
        batch_x, batch_y, idx = next_batch(train, batch_size = batch_size, index = idx)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y: batch_y})
            print("Step " + str(step) + ", Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:",sess.run(accuracy, feed_dict={X: test[0], Y: test[1]}))

Step 1, Loss= 11289.8096, Training Accuracy= 0.250
Step 50, Loss= 621.6454, Training Accuracy= 0.836
Step 100, Loss= 245.6350, Training Accuracy= 0.906
Step 150, Loss= 239.2593, Training Accuracy= 0.820
Step 200, Loss= 135.5943, Training Accuracy= 0.867
Step 250, Loss= 131.0241, Training Accuracy= 0.852
Step 300, Loss= 79.8466, Training Accuracy= 0.852
Step 350, Loss= 46.4432, Training Accuracy= 0.867
Step 400, Loss= 62.9294, Training Accuracy= 0.836
Step 450, Loss= 54.5932, Training Accuracy= 0.820
Step 500, Loss= 26.1929, Training Accuracy= 0.891
Optimization Finished!
Testing Accuracy: 0.8634


# 2. Bài tập

<b>Câu 1:</b> <br/>
<b>Iris flower dataset</b> là một bộ dữ liệu nhỏ (nhỏ hơn rất nhiều so với MNIST. Bộ dữ liệu này bao gồm thông tin của ba loại hoa Iris (một loài hoa lan) khác nhau: <i>Iris setosa, Iris virginica</i> và <i>Iris versicolor</i>. Mỗi loại có 50 bông hoa được đo với dữ liệu là 4 thông tin: chiều dài, chiều rộng đài hoa (sepal), và chiều dài, chiều rộng cánh hoa (petal). Dưới đây là ví dụ về hình ảnh của ba loại hoa. (Chú ý, đây không phải là bộ cơ sở dữ liệu ảnh như MNIST, mỗi điểm dữ liệu trong tập này chỉ là một vector 4 chiều).<br/>
<b>Iris flower dataset</b> có sẵn trong thư viện <b>scikit-learn</b>.

In [31]:
from sklearn import datasets
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target

In [33]:
iris_X[0]

array([5.1, 3.5, 1.4, 0.2])

In [34]:
iris_y[0]

0

Tách train data và test data:<br/>
Giả sử chúng ta muốn dùng 50 điểm dữ liệu cho test set, 100 điểm còn lại cho training set. Scikit-learn có một hàm số cho phép chúng ta ngẫu nhiên lựa chọn các điểm này, như sau:

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris_X, iris_y, test_size=50)
print(len(X_train))
print(len(X_test))

100
50


Thiết lập mạng neural thích hợp để phân loại hoa Iris trên.